## Step 18: 메모리 절약 모드

17단계에서는 파이썬에서 메모리를 관리하는 방법에 대해 알아보고, 순환 참조 문제를 해결하기 위해 Variable 클래스를 수정해보았다. 이번에는 DeZero의 메모리 사용을 개선하기 위해 불필요한 미분 결과를 삭제하는 구조와 '역전파가 필요 없는 경우용 모드'를 추가해보겠다.

### 18.1 필요 없는 미분값 삭제

먼저 DeZero의 역전파를 개선하도록 하겠다. 현재의 DeZero에서는 모든 변수가 자신의 미분값을 변수에 저장해두고 있다.

```python
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))

t = add(x0, x1)
y = add(x0, t)
y.backward()

print(y.grad, t.grad)
print(x0.grad, x1.grad)
```

\> 1.0 1.0 

\> 2.0 1.0

위에서 사용자가 계산을 위해 제공한 변수는 x0와 x1으로, 다른 변수 t와 y는 이들의 계산에 의해 만들어진다. 이때 y.backward()를 실행하여 미분을 진행하면 네 개 변수 모두에 대하여 미분 결과가 변수에 저장된다. 

그러나 보통 머신러닝에서 역전파를 통해 구하고 싶은 미분값은 말단 변수 x0, x1으로 중간 변수인 t와 y의 미분값은 필요로 하지 않는다. 따라서 이러한 중간 변수에 대한 미분값을 제거할 수 있도록 설정을 추가해주면 메모리 사용량을 줄이는 데에 도움이 될 것 같다.

```python
class Variable:
    ...
    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = PriorityQueue()
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.put(f)
                seen_set.add(f)

        add_func(self.creator)

        while not funcs.empty():
            f = funcs.get()
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad:
                for y in f.outputs:
                    y().grad = None
                    
    def cleargrad(self):
        self.grad = None
```

backward 메서드 인수에 retain_grad를 추가해주었다. 이를 통해 중간 변수의 미분 결과 저장 여부를 정할 수 있도록 하였다. True인 경우 지금까지처럼 모든 변수의 미분값을 저장하고, False인 경우 중간 변수의 미분값을 모두 None으로 재설정한다.

### 18.2 Function 클래스 복습

DeZero에서 미분을 수행하려면 순전파를 수행한 뒤 역전파해주면 된다. 역전파 시에는 순전파의 계산 결과가 필요하므로 순전파 단계에서 이를 기억해둔다.

```python
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        self.generation = max([x.generation for x in inputs])

        for output in outputs:
            output.set_creator(self)

        self.inputs = inputs # 결과값을 보관하는 부분
        self.outputs = [weakref.ref(output) for output in outputs]

        return outputs if len(outputs) > 1 else outputs[0]

    def __lt__(self, other):
        return self.generation > other.generation
```

이와 같이 함수는 입력을 inputs라는 **인스턴스 변수**로 참조한다. 그 결과 inputs가 참조하는 변수의 참조 카운트가 1 증가하고, 이에 따라 \_\_call__ 메서드에서 벗어난 뒤에도 메모리에 생존한다.

### 18.3 Config 클래스를 활용한 모드 전환

인스턴스 변수 inputs는 역전파 계산 시 사용된다. 다만 간혹 미분값이 필요 없는 연산을 수행할 때에는 이를 저장할 필요가 없고, 계산 연결 또한 만들 필요가 없다.

이처럼 순전파만 할 경우를 위해 DeZero를 개선해보도록 하자.

'역전파 활성 모드'와 '역전파 비활성 모드' 전환을 위해 Config 클래스를 이용하여 구현해보도록 하자.

```python
class Config:
    enable_backprop = True
```

enable_backprop이라는 Boolean 타입 프로퍼티만을 가지는 매우 단순한 구조의 클래스이다. 해당 속성은 이후 '역전파 진행 여부'를 설정하는데 이용할 것이며, 값이 True일 경우 '역전파 활성 모드'로 가정하겠다.

굳이 이런 단순한 구조를 클래스화한 이유는 '설정 데이터는 한 군데에만 존재하도록 한다'라는 법칙을 지키기 위한 것으로, 클래스는 하나만 존재할 수 있다는 특성을 이용한 것이다.

이를 인스턴스화하지 않고 클래스 그대로 사용할 경우 설정 데이터의 중복 없이 사용할 수 있다.

```python
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        if Config.enable_backprop:
            self.generation = max([x.generation for x in inputs])

            for output in outputs:
                output.set_creator(self)

            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs]

        return outputs if len(outputs) > 1 else outputs[0]

    def __lt__(self, other):
        return self.generation > other.generation
```

기존 Function 클래스에 if문을 추가하여 Config 클래스 내 enable_backprop 속성이 True일 때만 역전파가 실행되도록 하였다.

### 18.4 모드 전환

앞에서 만든 구조를 활용하면 다음과 같이 계산 단계에서 모드를 전환할 수 있다.

```python
Config.enable_backprop = True
x = Variable(np.ones((100,100,100)))
y = square(square(square(x)))
y.backward()

Config.enable_backprop = False
x = Variable(np.ones((100,100,100)))
y = square(square(square(x)))
# y.backward()
```

큰 다차원 배열 x에 대해 square 함수를 세 번 적용하였다. 이렇게 할 경우 원소별 제곱이 이루어지는데, 위쪽 계산에서는 enable_backprop이 True여서 중간 계산 결과가 모두 저장이 되는 탓에 메모리 소모가 많아진다.

반면 아래 코드에서는 enable_backprop을 False로 했기 때문에 중간 결과가 저장되지 않고 사용 후 즉시 삭제 되어 메모리 소모를 줄일 수 있다.

### 18.5 with 문을 활용한 모드 전환

이어서 모드 전환을 더 쉽게 해주는 구조를 만들어보도록 하자.

파이썬에는 with라고 하는, 후처리를 자동으로 수행하고자 할 때 사용할 수 있는 구문이 존재한다. 자원을 '획득'하고 '사용'한 이후에는 이를 '반납'하는 과정이 필요하다. 대표적으로 파일을 열고 닫는 open과 close가 존재한다.

with 문 없이 파일에 무언가를 쓰기 위해서는 다음처럼 작성해야 한다.

```python
f = open('file.txt', 'w')
f.write('hello world')
f.close()
```

이처럼 open()으로 파일을 열고, 작업을 수행한 후, close()로 파일을 닫는다. 이때 매번 close()를 하기는 귀찮기도 하고 실수로 잊기도 한다.

with문을 사용하면 이러한 수고와 실수를 줄여준다.

```python
with open('file.txt', 'w') as f:
    f.write('hello world')
```

인터프리터가 with 블록에 들어갈 때 파일이 열린다. 이후 블록 내 작업을 수행하고 최종적으로 블록을 빠져나올 때 with문에 의해 획득 된 자원이 자동으로 반환(close)된다.

이와 같이, with 문을 사용하는 것으로 'with 블록에 들어갈 때의 처리(전처리)'와 'with 블록을 빠져나올 때의 처리(후처리)'를 자동으로 할 수 있다.

이러한 with 문을 사용하여 '역전파 비활성 모드'를 손쉽게 설정할 수 있는 구조를 만들어보자.

```python
with using_config('enable_backprop', False):
    x = Variable(np.ones(2.0))
    y = square(x)
```

이와 같이 `with using_config('enable_backprop', False):` 안에 들어가면 '역전파 비활성 모드'로 전환된다. 이후 해당 블록을 벗어나면 일반 모드, 즉 '역전파 활성 모드'로 전환된다.

실제로 '역전파 비활성 모드'로의 일시적인 전환은 실전에서 자주 사용된다. 예를 들어 신경망 학습 단계에 있어 학습 도중에 모델 평가를 하는 경우 기울기가 필요 없는 모드를 일시적으로 사용하곤 한다.

contextlib 모듈을 사용하여 with 문 모드 전환을 구현해보도록 하자.

먼저 contextlib 모듈이 무엇인지 알아보도록 하자.

```python
import contextlib

@contextlib.contextmanager
def config_test():
    print('start') # 전처리
    try:
        yield
    finally:
        print('done') # 후처리

with config_test():
    print('process...')
```

\> start

\> process...

\> done

위처럼 @contextlib.contextmanager 데코레이터를 달면 문맥(context)을 판단하는 함수가 만들어진다. 그리고 이 함수 안에서 yield 전에는 전처리 로직을, yield 다음에는 후처리 로직을 작성한다. 이렇게 할 경우 with config_test(): 와 같이 사용 가능하며, 이를 사용하면 with 블록 안으로 들어갈 때 전처리가 실행되고 블록 범위를 빠져나올 때 후처리가 실행된다.

이때 with 블록 안에서 예외가 발생할 수 있으므로 yield는 try / finally로 감싸야 한다.

이를 바탕으로 using_config 함수를 다음과 같이 작성할 수 있다.

```python
import contextlib

@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)
```

using_config(name, value)의 인수 중 name은 타입이 str이며, 사용할 Config 속성의 이름(클래스 속성 이름)을 가리킨다.

이렇게 받은 클래스 속성 이름을 getattr 함수에 넘겨 Config 클래스에서 꺼내온다. 이후 setattr 함수를 이용해 새로운 값을 설정한다.

이제 with 블록에 들어갈 때 name으로 지정한 Config 클래스 속성이 value로 설정된다. 이후 with 블록을 빠져나올 때, getattr에 의해 가져와진 old_value에 의해 Config의 속성이 복원된다.

```python
with using_config('enable_backprop', False):
    x = Variable(np.array(2.0))
    y = square(x)
```

이와 같이 역전파가 필요 없는 경우에는 with 블록에서 순전파 코드만 실행된다. 이를 통해 불필요한 계산을 생략하고 메모리를 아낄 수 있다.

with using_config 구문을 적어주는 것도 귀찮으니 no_grad라는 helper function도 만들어보도록 하겠다.

```python
def no_grad():
    return using_config('enable_backprop', False)

with no_grad():
    x = Variable(np.array(2.0))
    y = square(x)
```